# **ASL Pepper**

# **Carga del dataset**

Se conecta el drive al colab para poder sacar las imagenes del dataset y trabajar con ellas.

In [ ]:
import os
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math

from google.colab import drive
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from matplotlib.colors import ListedColormap

In [ ]:
drive.mount('/content/gdrive', force_remount=True)

Se descarga el dataset directamente de Kaggle.

In [ ]:
os.environ['KAGGLE_CONFIG_DIR'] = "/content/gdrive/My Drive/token/"
#/content/gdrive/My Drive/token/ is the path where kaggle.json is present in the Google Drive

#changing the working directory
%cd /content/

#downloading the dataset
!kaggle datasets download -d grassknoted/asl-alphabet
!ls
!unzip \*.zip  && rm *.zip


Confirmando que exista el directorio 

In [ ]:
!ls

Se procede a cargar las imagenes del directorio de entrenamiento y dividirlo en 2 subconjuntos, el conjunto de entrenamiento y el conjunto de validación.

In [5]:
#Constants
LR = 0.0001
EPOCHS = 15
BATCH_SIZE = 64

In [6]:
train_dir = r'/content/asl_alphabet_train/asl_alphabet_train/'

unique_labels = sorted(os.listdir(train_dir))
print(unique_labels)

data_gen=ImageDataGenerator(
          rescale = 1./255.,
          validation_split=0.2)

train_gen = data_gen.flow_from_directory(train_dir, target_size=(128, 128), color_mode = 'grayscale', class_mode='categorical', batch_size=BATCH_SIZE, subset='training')
validation_gen = data_gen.flow_from_directory(train_dir, target_size=(128, 128), color_mode = 'grayscale', class_mode='categorical', batch_size=BATCH_SIZE, subset='validation')

'''
x,y= train_gen.next()
for i in range(0,64):
    image = x[i].reshape((128,128))
    plt.imshow(image, cmap = 'gray', vmin = 0, vmax = 255)
    plt.show()
'''

['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'del', 'nothing', 'space']
Found 69600 images belonging to 29 classes.
Found 17400 images belonging to 29 classes.


"\nx,y= train_gen.next()\nfor i in range(0,64):\n    image = x[i].reshape((128,128))\n    plt.imshow(image, cmap = 'gray', vmin = 0, vmax = 255)\n    plt.show()\n"

# **Creación y entrenamiento de la red neuronal**

### **Creación de la red neuronal**

In [ ]:
model = tf.keras.models.Sequential([
        tf.keras.layers.Conv2D(filters = 32, kernel_size = (2,2), strides = (2,2), activation = 'relu', input_shape = (128,128,1)),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.MaxPooling2D(pool_size = (1,1), strides = (2,2)),
        tf.keras.layers.Conv2D(filters = 32, kernel_size = (2,2), strides = (2,2), activation = 'relu'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.MaxPooling2D(pool_size = (1,1), strides = (2,2)),
        tf.keras.layers.Conv2D(filters = 64, kernel_size = (2,2), strides = (2,2), activation = 'relu'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.MaxPooling2D(pool_size = (1,1), strides = (2,2)),
        tf.keras.layers.Conv2D(filters = 64, kernel_size = (2,2), strides = (2,2), activation = 'relu'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.AveragePooling2D(pool_size = (1,1), strides = (2,2)),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(units = 128, activation = 'relu'),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(units = 29, activation = 'softmax')
])
model.summary()

### **Entrenamiento de la red neuronal**

In [12]:
from keras import backend as K
def recall(y_true, y_pred):
    y_true = K.ones_like(y_true) 
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    all_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    
    recall_v = true_positives / (all_positives + K.epsilon())
    return recall_v

def precision(y_true, y_pred):
    y_true = K.ones_like(y_true) 
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision_v = true_positives / (predicted_positives + K.epsilon())
    return precision_v

def f1_scorer(y_true, y_pred):
    precision_m = precision(y_true, y_pred)
    recall_m = recall(y_true, y_pred)
    return 2*((precision_m*recall_m)/(precision_m+recall_m+K.epsilon()))

In [13]:
adam_opt = Adam(learning_rate = LR)

model.compile(optimizer = adam_opt, loss = 'categorical_crossentropy', metrics = ['accuracy', f1_scorer, precision, recall])

In [ ]:
model.fit(x = train_gen,
          validation_data = validation_gen,
          steps_per_epoch = math.ceil(train_gen.n / BATCH_SIZE),
          epochs = EPOCHS,
          validation_steps = math.ceil(validation_gen.n / BATCH_SIZE),
          verbose = 1)

## **Almacenamiento del modelo**

In [ ]:
model.save('/content/gdrive/My Drive/Token/my_model.h5')

# **Predicción del conjunto de pruebas y medición del desempeño**

### **Classification report para el conjunto de entrenamiento**

In [15]:
labels = []
for i in range(0, 29):
  labels.append(i)

In [ ]:
y_true_train = train_gen.classes
y_pred_train = model.predict(train_gen)
y_pred_max_train = np.argmax(y_pred_train, axis=1)

train_score = classification_report(y_true = y_true_train, y_pred = y_pred_max_train, labels = labels, target_names = unique_labels)

print(train_score)

### **Classification report para el conjunto de pruebas**

In [ ]:
y_true_test = validation_gen.classes
y_pred_test = model.predict(validation_gen)
y_pred_max_test = np.argmax(y_pred_test, axis=1)

test_score = classification_report(y_true = y_true_test, y_pred = y_pred_max_test, labels = labels, target_names = unique_labels)

print(test_score)

In [18]:
colors = ["#FFFFFF"]
customPalette = sns.set_palette(sns.color_palette(colors))

### **Confusion matrix para el conjunto de entrenamiento**

In [ ]:
c_matrix_train = confusion_matrix(y_true_train, y_pred_max_train, labels=labels)

plt.figure(figsize = (20,20))
sns.heatmap(c_matrix_train, annot=True, fmt = "d", cmap = ListedColormap(['white']), linewidths = 0.01, linecolor = "black", cbar = False, square = True, xticklabels = unique_labels, yticklabels = unique_labels)
plt.show()

### **Confusion matrix para el conjunto de pruebas**

In [ ]:
c_matrix_test = confusion_matrix(y_true_test, y_pred_max_test, labels=labels)

plt.figure(figsize = (20,20))
sns.heatmap(c_matrix_test, annot=True, fmt = "d", cmap = ListedColormap(['white']), linewidths = 0.01, linecolor = "black", cbar = False, square = True, xticklabels = unique_labels, yticklabels = unique_labels)
plt.show()